In [1]:
import numpy as np
import pandas as pd
#from pandas_profiling import ProfileReport
import glob
from datetime import datetime
import time
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, mean_squared_error,median_absolute_error,mean_absolute_error
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.svm import LinearSVC,LinearSVR
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor, MLPClassifier
import matplotlib.pyplot as plt
from itertools import permutations
import numpy as np
from itertools import combinations 
%matplotlib inline
import xgboost as xg

import seaborn as sns
sns.set()
import numpy

In [2]:
state_data = pd.DataFrame()

state_csv = 'state.csv'
state_data = pd.read_csv(state_csv, low_memory=False)
state_data.head()

,date,state_code,gender,age_bucket,n,weight_sums,pct_cli,pct_ili,pct_cli_anosmia_ageusia,pct_hh_cli,...,pct_diabetes_weighted,pct_cancer_weighted,pct_heart_disease_weighted,pct_high_blood_pressure_weighted,pct_asthma_weighted,pct_chronic_lung_disease_weighted,pct_kidney_disease_weighted,pct_autoimmune_disorder_weighted,pct_no_above_medical_conditions_weighted,pct_multiple_medical_conditions_weighted
0,2020-04-06,ak,female,18-34,119,78755.310877,0.8772,0.8772,2.6316,0.8403,...,3.1304,0.0000,0.0000,7.1685,28.7123,0.0000,0.0000,9.3777,63.1666,10.3406
1,2020-04-06,ak,female,35-54,95,63666.933392,3.2258,3.2258,3.2258,3.1579,...,9.7745,2.8142,6.9796,33.3235,29.5650,1.4395,0.0000,11.1606,46.8402,25.1121
2,2020-04-06,ak,female,55+,51,52710.069695,0.0000,0.0000,1.9608,0.0000,...,6.9947,19.3862,1.9712,26.1060,18.2128,6.3439,0.0000,8.1722,35.4451,22.9556
3,2020-04-06,al,female,18-34,478,480481.152573,0.8547,1.0684,2.1368,1.6771,...,3.1854,0.1110,1.1657,10.9676,21.6862,0.4720,1.5429,5.4713,62.7683,6.3179
4,2020-04-06,al,female,35-54,482,557081.407562,1.4706,1.6807,2.3109,2.2869,...,12.1389,3.1155,5.4066,34.8936,14.9863,5.8535,2.2455,14.0900,42.6328,24.1615


In [3]:
state_data_dropped = state_data.dropna()
state_data_dropped = state_data_dropped[(state_data_dropped['gender']=='overall') & (state_data_dropped['age_bucket']=='overall')]

In [4]:
y= state_data_dropped['pct_tested_and_positive']
output_like_variables=['pct_tested_and_positive','pct_tested_and_negative', 'pct_tested_no_result', 'pct_could_not_get_tested', 'pct_did_not_try_to_get_tested',\
                       'pct_tested_and_positive_weighted','pct_contact_covid_positive', 'pct_contact_covid_positive_weighted' ,\
                       'pct_tested_and_negative_weighted',	'pct_tested_no_result_weighted',	'pct_could_not_get_tested_weighted',	'pct_did_not_try_to_get_tested_weighted'] 
                        
X=state_data_dropped.drop(output_like_variables,axis=1)
X.columns

Index(['date', 'state_code', 'gender', 'age_bucket', 'n', 'weight_sums',
       'pct_cli', 'pct_ili', 'pct_cli_anosmia_ageusia', 'pct_hh_cli',
       'pct_cmnty_cli', 'pct_hh_fever', 'pct_hh_sore_throat', 'pct_hh_cough',
       'pct_hh_shortness_of_breath', 'pct_hh_difficulty_breathing',
       'mean_hh_cli_ct', 'mean_cmnty_cli_ct', 'pct_self_fever',
       'pct_self_cough', 'pct_self_shortness_of_breath',
       'pct_self_difficulty_breathing', 'pct_self_tiredness_or_exhaustion',
       'pct_self_nasal_congestion', 'pct_self_runny_nose',
       'pct_self_muscle_joint_aches', 'pct_self_sore_throat',
       'pct_self_persistent_pain_pressure_in_chest',
       'pct_self_nausea_vomiting', 'pct_self_diarrhea',
       'pct_self_anosmia_ageusia', 'pct_self_other', 'pct_self_none_of_above',
       'pct_self_multiple_symptoms', 'pct_worked_outside_home',
       'pct_avoid_contact_all_or_most_time',
       'mean_outside_hh_contact_at_work_ct',
       'mean_outside_hh_contact_shopping_ct',
     

In [5]:
unweighted=['n','weight_sums','pct_cli','pct_ili', 'pct_cli_anosmia_ageusia',\
        'pct_hh_cli', 'pct_cmnty_cli', 'pct_hh_fever', 'pct_hh_sore_throat',\
         'pct_hh_cough', 'pct_hh_shortness_of_breath','pct_hh_difficulty_breathing', 'mean_hh_cli_ct', 'mean_cmnty_cli_ct',\
         'pct_self_fever', 'pct_self_cough', 'pct_self_shortness_of_breath',\
         'pct_self_difficulty_breathing', 'pct_self_tiredness_or_exhaustion',\
         'pct_self_nasal_congestion', 'pct_self_runny_nose',\
         'pct_self_muscle_joint_aches', 'pct_self_sore_throat',
       'pct_self_persistent_pain_pressure_in_chest',\
       'pct_self_nausea_vomiting', 'pct_self_diarrhea',\
       'pct_self_anosmia_ageusia', 'pct_self_other', 'pct_self_none_of_above',\
       'pct_self_multiple_symptoms', 'pct_worked_outside_home',\
       'pct_avoid_contact_all_or_most_time',\
       'mean_outside_hh_contact_at_work_ct',\
       'mean_outside_hh_contact_shopping_ct',\
       'mean_outside_hh_contact_in_social_gatherings_ct', 'pct_diabetes',\
       'pct_cancer', 'pct_heart_disease', 'pct_high_blood_pressure', \
       'pct_asthma', 'pct_chronic_lung_disease', 'pct_kidney_disease',\
       'pct_autoimmune_disorder', 'pct_no_above_medical_conditions',\
       'pct_multiple_medical_conditions', ]
X=X.drop(unweighted,axis=1)
n_features = X.shape[1]
print(n_features)

47


In [6]:
X.columns

Index(['date', 'state_code', 'gender', 'age_bucket', 'pct_cli_weighted',
       'pct_ili_weighted', 'pct_cli_anosmia_ageusia_weighted',
       'pct_hh_cli_weighted', 'pct_cmnty_cli_weighted',
       'pct_hh_fever_weighted', 'pct_hh_sore_throat_weighted',
       'pct_hh_cough_weighted', 'pct_hh_shortness_of_breath_weighted',
       'pct_hh_difficulty_breathing_weighted', 'mean_hh_cli_ct_weighted',
       'mean_cmnty_cli_ct_weighted', 'pct_self_fever_weighted',
       'pct_self_cough_weighted', 'pct_self_shortness_of_breath_weighted',
       'pct_self_difficulty_breathing_weighted',
       'pct_self_tiredness_or_exhaustion_weighted',
       'pct_self_nasal_congestion_weighted', 'pct_self_runny_nose_weighted',
       'pct_self_muscle_joint_aches_weighted', 'pct_self_sore_throat_weighted',
       'pct_self_persistent_pain_pressure_in_chest_weighted',
       'pct_self_nausea_vomiting_weighted', 'pct_self_diarrhea_weighted',
       'pct_self_anosmia_ageusia_weighted', 'pct_self_other_weighte

In [7]:
derived=['pct_ili_weighted','pct_cli_weighted','pct_cli_anosmia_ageusia_weighted',]
X=X.drop(derived,axis=1)
n_features = X.shape[1]
print(n_features)

44


In [8]:
mean_D = ['mean_outside_hh_contact_in_social_gatherings_ct_weighted','mean_cmnty_cli_ct_weighted',\
        'mean_outside_hh_contact_shopping_ct_weighted','mean_outside_hh_contact_at_work_ct_weighted']
X=X.drop(mean_D,axis=1)
n_features = X.shape[1]
print(n_features)

40


In [9]:
who = pd.read_csv('refined-us-states.csv')
who.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13764 entries, 0 to 13763
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    13764 non-null  object
 1   state   13764 non-null  object
 2   cases   13764 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 322.7+ KB


In [10]:
who.head()

,date,state,cases
0,13-03-2020,al,6
1,14-03-2020,al,12
2,15-03-2020,al,23
3,16-03-2020,al,29
4,17-03-2020,al,39


In [11]:
who = who.rename(columns={"state": "state_code"})

In [12]:
who['date'] = list(map(lambda x : '-'.join(list(reversed(x.split('-')))) ,list(who['date'])) )
who.head()

,date,state_code,cases
0,2020-03-13,al,6
1,2020-03-14,al,12
2,2020-03-15,al,23
3,2020-03-16,al,29
4,2020-03-17,al,39


In [13]:
merged = pd.merge(X, who,  how='inner', on=['date','state_code'] )
merged.head()

,date,state_code,gender,age_bucket,pct_hh_cli_weighted,pct_cmnty_cli_weighted,pct_hh_fever_weighted,pct_hh_sore_throat_weighted,pct_hh_cough_weighted,pct_hh_shortness_of_breath_weighted,...,pct_cancer_weighted,pct_heart_disease_weighted,pct_high_blood_pressure_weighted,pct_asthma_weighted,pct_chronic_lung_disease_weighted,pct_kidney_disease_weighted,pct_autoimmune_disorder_weighted,pct_no_above_medical_conditions_weighted,pct_multiple_medical_conditions_weighted,cases
0,2020-04-16,ak,overall,overall,0.4520,15.9583,0.4520,5.8922,15.9674,4.1607,...,2.8519,7.3959,30.9819,19.2527,1.4890,0.0000,5.6824,46.5078,19.1625,298
1,2020-04-16,al,overall,overall,1.6437,17.2337,2.0935,7.9069,19.6641,6.6038,...,4.9368,7.6839,38.5953,13.4155,5.9090,1.3226,7.0919,43.1997,23.8771,4404
2,2020-04-16,ar,overall,overall,0.8254,15.5431,1.0411,8.3399,23.8985,8.7985,...,4.6709,12.1418,36.0491,12.7897,6.2749,2.6248,6.6227,47.5899,28.4986,1620
3,2020-04-16,az,overall,overall,1.8524,12.5070,2.2566,7.3155,15.9989,6.5837,...,5.8572,6.6829,32.6985,17.9660,5.7625,2.3895,6.6866,42.8403,22.6322,4234
4,2020-04-16,ca,overall,overall,1.0853,11.8908,1.6494,7.7404,15.2963,5.0223,...,4.9180,4.5262,25.4755,16.0995,3.3386,1.9608,5.5005,52.3355,17.1472,28142


In [14]:
merged = merged.sort_values(['state_code', 'date'], ascending=[True, True])

In [15]:
merged.head()

,date,state_code,gender,age_bucket,pct_hh_cli_weighted,pct_cmnty_cli_weighted,pct_hh_fever_weighted,pct_hh_sore_throat_weighted,pct_hh_cough_weighted,pct_hh_shortness_of_breath_weighted,...,pct_cancer_weighted,pct_heart_disease_weighted,pct_high_blood_pressure_weighted,pct_asthma_weighted,pct_chronic_lung_disease_weighted,pct_kidney_disease_weighted,pct_autoimmune_disorder_weighted,pct_no_above_medical_conditions_weighted,pct_multiple_medical_conditions_weighted,cases
0,2020-04-16,ak,overall,overall,0.4520,15.9583,0.452,5.8922,15.9674,4.1607,...,2.8519,7.3959,30.9819,19.2527,1.4890,0.0000,5.6824,46.5078,19.1625,298
51,2020-04-17,ak,overall,overall,0.6280,13.7019,0.628,4.7968,12.4514,4.9752,...,3.6710,3.4297,21.9018,15.7770,2.6192,0.8145,7.0081,56.7640,15.2023,307
101,2020-04-18,ak,overall,overall,0.0000,17.1224,0.000,4.0468,17.1023,2.9509,...,1.9051,6.4604,33.7803,19.2675,1.4961,2.5417,4.8131,39.3180,11.5682,312
151,2020-04-19,ak,overall,overall,0.0000,11.5773,0.000,6.5950,16.4129,9.6173,...,4.5857,7.1246,31.8834,17.6192,7.0559,1.8107,5.5353,46.0464,16.3545,317
201,2020-04-20,ak,overall,overall,1.0562,16.4867,2.112,5.4416,10.4745,7.1325,...,5.0504,7.4312,29.1596,17.5441,6.1758,1.4354,7.2230,44.9073,15.2334,319


In [16]:
print(len(merged))
merged['state_code'].nunique()

7363


50

In [17]:
merged = merged.drop_duplicates(subset = ['state_code','date'])
len(merged)

7228

In [18]:
merged['daily_case'] = merged['cases'].diff().fillna(merged['cases'].iloc[0])

In [19]:
merged.head()

,date,state_code,gender,age_bucket,pct_hh_cli_weighted,pct_cmnty_cli_weighted,pct_hh_fever_weighted,pct_hh_sore_throat_weighted,pct_hh_cough_weighted,pct_hh_shortness_of_breath_weighted,...,pct_heart_disease_weighted,pct_high_blood_pressure_weighted,pct_asthma_weighted,pct_chronic_lung_disease_weighted,pct_kidney_disease_weighted,pct_autoimmune_disorder_weighted,pct_no_above_medical_conditions_weighted,pct_multiple_medical_conditions_weighted,cases,daily_case
0,2020-04-16,ak,overall,overall,0.4520,15.9583,0.452,5.8922,15.9674,4.1607,...,7.3959,30.9819,19.2527,1.4890,0.0000,5.6824,46.5078,19.1625,298,298.0
51,2020-04-17,ak,overall,overall,0.6280,13.7019,0.628,4.7968,12.4514,4.9752,...,3.4297,21.9018,15.7770,2.6192,0.8145,7.0081,56.7640,15.2023,307,9.0
101,2020-04-18,ak,overall,overall,0.0000,17.1224,0.000,4.0468,17.1023,2.9509,...,6.4604,33.7803,19.2675,1.4961,2.5417,4.8131,39.3180,11.5682,312,5.0
151,2020-04-19,ak,overall,overall,0.0000,11.5773,0.000,6.5950,16.4129,9.6173,...,7.1246,31.8834,17.6192,7.0559,1.8107,5.5353,46.0464,16.3545,317,5.0
201,2020-04-20,ak,overall,overall,1.0562,16.4867,2.112,5.4416,10.4745,7.1325,...,7.4312,29.1596,17.5441,6.1758,1.4354,7.2230,44.9073,15.2334,319,2.0


In [20]:
print(len(merged))
merged = merged[ ~merged['date'].isin(['2020-04-16'])]
print(len(merged))

7228
7178


In [21]:
merged.columns

Index(['date', 'state_code', 'gender', 'age_bucket', 'pct_hh_cli_weighted',
       'pct_cmnty_cli_weighted', 'pct_hh_fever_weighted',
       'pct_hh_sore_throat_weighted', 'pct_hh_cough_weighted',
       'pct_hh_shortness_of_breath_weighted',
       'pct_hh_difficulty_breathing_weighted', 'mean_hh_cli_ct_weighted',
       'pct_self_fever_weighted', 'pct_self_cough_weighted',
       'pct_self_shortness_of_breath_weighted',
       'pct_self_difficulty_breathing_weighted',
       'pct_self_tiredness_or_exhaustion_weighted',
       'pct_self_nasal_congestion_weighted', 'pct_self_runny_nose_weighted',
       'pct_self_muscle_joint_aches_weighted', 'pct_self_sore_throat_weighted',
       'pct_self_persistent_pain_pressure_in_chest_weighted',
       'pct_self_nausea_vomiting_weighted', 'pct_self_diarrhea_weighted',
       'pct_self_anosmia_ageusia_weighted', 'pct_self_other_weighted',
       'pct_self_none_of_above_weighted',
       'pct_self_multiple_symptoms_weighted',
       'pct_worked_ou

In [22]:
merged = merged.drop(['mean_hh_cli_ct_weighted'], axis =1)
len(merged.columns)

41

In [23]:
features = ['pct_cmnty_cli_weighted', 'pct_self_anosmia_ageusia_weighted', 'pct_hh_cli_weighted', 'pct_hh_fever_weighted', 'pct_self_fever_weighted', 'pct_hh_sore_throat_weighted', 'pct_avoid_contact_all_or_most_time_weighted', 'pct_hh_difficulty_breathing_weighted', 'pct_self_persistent_pain_pressure_in_chest_weighted', 'pct_self_runny_nose_weighted', 'pct_worked_outside_home_weighted', 'pct_self_nausea_vomiting_weighted', 'pct_hh_shortness_of_breath_weighted', 'pct_self_sore_throat_weighted', 'pct_self_difficulty_breathing_weighted', 'pct_asthma_weighted', 'pct_self_shortness_of_breath_weighted', 'pct_hh_cough_weighted', 'pct_self_none_of_above_weighted', 'pct_self_diarrhea_weighted', 'pct_chronic_lung_disease_weighted', 'pct_cancer_weighted', 'pct_self_other_weighted', 'pct_self_tiredness_or_exhaustion_weighted', 'pct_self_cough_weighted', 'pct_no_above_medical_conditions_weighted', 'pct_heart_disease_weighted', 'pct_multiple_medical_conditions_weighted', 'pct_autoimmune_disorder_weighted', 'pct_self_nasal_congestion_weighted', 'pct_kidney_disease_weighted',  'pct_self_multiple_symptoms_weighted', 'pct_self_muscle_joint_aches_weighted', 'pct_high_blood_pressure_weighted', 'pct_diabetes_weighted',  ]


In [24]:
len(features)

35

In [25]:
merged.to_csv('state_with_real_numbers.csv',index=False)

In [26]:
merged.head()

,date,state_code,gender,age_bucket,pct_hh_cli_weighted,pct_cmnty_cli_weighted,pct_hh_fever_weighted,pct_hh_sore_throat_weighted,pct_hh_cough_weighted,pct_hh_shortness_of_breath_weighted,...,pct_heart_disease_weighted,pct_high_blood_pressure_weighted,pct_asthma_weighted,pct_chronic_lung_disease_weighted,pct_kidney_disease_weighted,pct_autoimmune_disorder_weighted,pct_no_above_medical_conditions_weighted,pct_multiple_medical_conditions_weighted,cases,daily_case
51,2020-04-17,ak,overall,overall,0.6280,13.7019,0.6280,4.7968,12.4514,4.9752,...,3.4297,21.9018,15.7770,2.6192,0.8145,7.0081,56.7640,15.2023,307,9.0
101,2020-04-18,ak,overall,overall,0.0000,17.1224,0.0000,4.0468,17.1023,2.9509,...,6.4604,33.7803,19.2675,1.4961,2.5417,4.8131,39.3180,11.5682,312,5.0
151,2020-04-19,ak,overall,overall,0.0000,11.5773,0.0000,6.5950,16.4129,9.6173,...,7.1246,31.8834,17.6192,7.0559,1.8107,5.5353,46.0464,16.3545,317,5.0
201,2020-04-20,ak,overall,overall,1.0562,16.4867,2.1120,5.4416,10.4745,7.1325,...,7.4312,29.1596,17.5441,6.1758,1.4354,7.2230,44.9073,15.2334,319,2.0
252,2020-04-21,ak,overall,overall,3.1658,12.1468,5.0526,10.6019,11.5072,8.3987,...,6.3267,29.3964,21.0547,1.5149,2.2544,7.6099,42.4108,20.7492,327,8.0
